In [36]:
from selenium import webdriver
import os
import time
import requests
import io
from PIL import Image
import hashlib

In [37]:

# This is the path I use
DRIVER_PATH = 'chromedriver_mac/chromedriver'
# Put the path for your ChromeDriver here
#DRIVER_PATH = <YOUR PATH>
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

/var/folders/tg/8cb9xv716j908pl8qmdbqnj40000gn/T/ipykernel_3403/1117134011.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [55]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                print("Found:", len(image_urls), "image links, looking for more ...")
#                 time.sleep(5)
#                 continue
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")

                # move the result startpoint further down
                results_start = len(thumbnail_results)

    return image_urls

In [56]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [57]:
def search_and_download(search_term:str,driver_path:str,target_path='../images',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [58]:
search_term = 'steamed egg'

search_and_download(
    search_term = search_term,
    driver_path =DRIVER_PATH
)

/var/folders/tg/8cb9xv716j908pl8qmdbqnj40000gn/T/ipykernel_3403/2552248140.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
/var/folders/tg/8cb9xv716j908pl8qmdbqnj40000gn/T/ipykernel_3403/1432786489.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


/var/folders/tg/8cb9xv716j908pl8qmdbqnj40000gn/T/ipykernel_3403/1432786489.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
/var/folders/tg/8cb9xv716j908pl8qmdbqnj40000gn/T/ipykernel_3403/1432786489.py:47: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  load_more_button = wd.find_element_by_css_selector(".mye4qd")


Found: 1 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 2 image links, looking for more ...
Found: 3 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 4 image links, looking for more ...
Found: 5 image links, looking for more ...
Found: 6 image links, looking for more ...
Found: 7 image links, looking for more ...
Found: 8 image links, looking for more ...
Found: 9 image links, looking for more ...
Found: 10 image links, looking for more ...
Found: 11 image links, looking for more ...
Found: 11 image links, looking for more ...
Found: 12 image links, looking for more ...
Found: 12 image links, looking for more ...
Found: 13 image links, looking for more ...
Found: 14 image links, looking for more ...
Found: 15 image links, looking for more ...
Found: 17 image links, looking for more ...
Found: 19 image links, looking for more ...
Found: 21 image links, looking for more ...
Found: 22 image links, looking for more ...

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQD9TS-oiPdG3mKRhvRpDwNfDin7HK-rlRbbw&usqp=CAU - as ../images/steamed_egg/f6b4bc5ac9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSo0FuhmYbVM1tyxo5XnF6ysmSHcINugfEeXA&usqp=CAU - as ../images/steamed_egg/1f8590c167.jpg
SUCCESS - saved https://whattocooktoday.com/wp-content/uploads/2011/12/chawanmushi-6.jpg - as ../images/steamed_egg/05c632e578.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwP6bOUG4BMVlbnKBANk6tOETQBGwh57QmYg&usqp=CAU - as ../images/steamed_egg/df717065d6.jpg
SUCCESS - saved https://media.eggs.ca/assets/RecipePhotos/_resampled/FillWyIxMjgwIiwiNzIwIl0/Steamed-egg-filled-with-minced-pork-and-conpoy-CMS.jpg - as ../images/steamed_egg/061aaad276.jpg
SUCCESS - saved https://kirbiecravings.com/wp-content/uploads/2019/10/microwave-steamed-eggs-4.jpg - as ../images/steamed_egg/ec7ec6734a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:

SUCCESS - saved https://images.squarespace-cdn.com/content/v1/542e0ddce4b03f0fd61be184/1469980280787-3QVO6IHCBNVYFQG2G8W6/image-asset.jpeg - as ../images/steamed_egg/630ab9bfdf.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQ9SM3TtRXmnl8J82rllBX3jm23_xgSEIcCw&usqp=CAU - as ../images/steamed_egg/de18a2e948.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxoAaplt_DUQyJCPK9EYxZ_pj5-c9e-y37cw&usqp=CAU - as ../images/steamed_egg/2535c0971b.jpg
SUCCESS - saved http://images.summitmedia-digital.com/cosmo/images/2020/06/27/korean-steamed-egg-1593215879.jpg - as ../images/steamed_egg/1efdf0da64.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT_87RssNxW-QB0h6o0YoGArzPEsin5rbbIjg&usqp=CAU - as ../images/steamed_egg/3f35c454eb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTUiM3GT3qv-rKj3R0t8Y9dAGzWo0fXEGW_A&usqp=CAU - as ../images/steamed_egg/b265e8faec.jpg
SUCCESS - saved https://encrypted

SUCCESS - saved https://i.ytimg.com/vi/7r14gGvGIK8/maxresdefault.jpg - as ../images/steamed_egg/82f4bc9087.jpg
SUCCESS - saved https://www.wokandkin.com/wp-content/uploads/2020/07/Chinese-Steamed-Eggs-saved-for-web.png - as ../images/steamed_egg/1057719dd7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjNX355HH4QSWeRUJ2asBIZWCzPErZMWaGXA&usqp=CAU - as ../images/steamed_egg/744f986ad7.jpg
SUCCESS - saved https://www.sbs.com.au/food/sites/sbs.com.au.food/files/Korean-steamed-egg.jpg - as ../images/steamed_egg/dfd2b6f905.jpg
SUCCESS - saved https://redhousespice.com/wp-content/uploads/2017/10/Chinese-steamed-eggs-portrait1.jpg - as ../images/steamed_egg/fc12fd8705.jpg
SUCCESS - saved https://whattocooktoday.com/wp-content/uploads/2021/08/steamed-egg-minced-pork-10.jpg - as ../images/steamed_egg/ee700f39d9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTTzZqCsiXmuCNM7gTKL4H6JuOZPiC5JzaPvg&usqp=CAU - as ../images/steamed_egg/976a842607.

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_6sQO-j2DRHbGX_-npjWje3yVmybAqBbQnA&usqp=CAU - as ../images/steamed_egg/5b74880456.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyanpf7pRXtfuYUQH9J5e9f8qFk4EX0ufPlg&usqp=CAU - as ../images/steamed_egg/02d64b0ffd.jpg
SUCCESS - saved https://static01.nyt.com/images/2021/09/29/dining/EK-microwave-steamed-eggs/merlin_195007317_11c416b4-5607-44f7-8781-de6af1c64076-articleLarge.jpg - as ../images/steamed_egg/82bc6c6d91.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFsD5M7Hy1lrb44QFalbee-uO3gV80IoKewA&usqp=CAU - as ../images/steamed_egg/092a8b0714.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSttSnheC_UFuktDF2YbB8nFeVR95Duo3ANsA&usqp=CAU - as ../images/steamed_egg/2363b9d51b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR3vguuVqs8w3YA6WklvxDiPA6I2F1F8a5Nvg&usqp=CAU - as ../images/steamed_egg/43593017a8.jpg
SUCCE